![miad4.png](Archivos/miad4.png)

# Interactividad utilizando `panel`

`panel` es una librería de Python que nos permite crear aplicaciones web interactivas y tableros de control personalizados mediante la conexión de controles definidos por el usuario con gráficos, imágenes, tablas o texto.

En este tutorial establecemos nociones fundamentales sobre el uso de la libería `panel` para integrarla en procesos de visualización de datos.

## Requisitos

Para desarrollar este tutorial necesitarás:
* Implementar estructuras de control.
* Utilizar las librerías `pandas` y `plotly`.

## Objetivos

Al final de este tutorial podrás:

**1.** Definir los componentes necesarios para crear un tablero de control. <br> 
**2.** Personalizar tableros de control con gráficos y controles para generar distintas visualizaciones interactivas.

## 1. Librería `panel`

### 1.1. Componentes necesarios

Para comenzar, debemos importar las librerías necesarias para crear tableros de control (*dashboards*). Te recomendamos importar la librería `panel` utilizando el nombre `pn` (su alias más frecuentemente usado), como se muestra a continuación.

In [1]:
import panel as pn
import pandas as pd
import plotly.graph_objects as go

Ahora, declaramos la extensión de `plotly` en `panel`.

In [2]:
pn.extension('plotly')

Luego, importamos la base de datos de los viajes en Citi Bike dentro de Nueva York durante algunos días del mes de enero de 2021 y realizamos una breve exploración de esta.

In [3]:
datos = pd.read_csv('Archivos/202101-citibike-tripdata.csv')
datos = pd.DataFrame(datos)
datos.head()

,Unnamed: 0.1,Unnamed: 0,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,0,0,2513,2021-01-01 00:00:11.9020,2021-01-01 00:42:05.2260,3581,Underhill Ave & Lincoln Pl,40.674012,-73.967146,3581,Underhill Ave & Lincoln Pl,40.674012,-73.967146,47812,Customer,1969,0
1,1,1,2519,2021-01-01 00:00:15.0960,2021-01-01 00:42:14.9780,3581,Underhill Ave & Lincoln Pl,40.674012,-73.967146,3581,Underhill Ave & Lincoln Pl,40.674012,-73.967146,47571,Customer,1969,0
2,2,2,1207,2021-01-01 00:00:28.9300,2021-01-01 00:20:36.6510,3144,E 81 St & Park Ave,40.776777,-73.959010,3724,7 Ave & Central Park South,40.766741,-73.979069,37451,Subscriber,2002,1
3,3,3,2506,2021-01-01 00:00:32.7130,2021-01-01 00:42:19.3980,3581,Underhill Ave & Lincoln Pl,40.674012,-73.967146,3581,Underhill Ave & Lincoln Pl,40.674012,-73.967146,48884,Customer,2002,1
4,4,4,959,2021-01-01 00:00:35.3650,2021-01-01 00:16:34.6010,534,Water - Whitehall Plaza,40.702551,-74.012723,332,Cherry St,40.712199,-73.979481,26837,Customer,2002,1


Las variables que utlizaremos se encuentran en la siguiente tabla:

| <center>Variable<center> | <center>Descripción<center> | 
| :--- | :--- | 
| `start station name` | Estación de inicio del viaje |
| `end station name` | Estación de llegada del viaje |
| `bikeid` | Indentificador de la bicicleta |

A continuación, definimos los componentes del tablero de control. Primero, agregamos un título.

In [4]:
titulo = '# Tablero de control Citi Bike'

Segundo, creamos una lista desplegable con las estaciones de salida.

In [5]:
seleccion = pn.widgets.Select(name='Estación de salida',
                              options=datos['start station name'].unique().tolist())

### 1.2. Personalización

A continuación, agregamos funcionalidad al tablero de control. 

In [6]:
# Le decimos a panel que nuestro gráfico depende de cambios en el párametro seleccion.
@pn.depends(seleccion.param.value)

# Definimos la función que se encarga de la interactividad del tablero de control.
def funcion_interactiva(seleccion): 

    # Cargamos los datos.
    df = datos 
    # Creamos un filtro de datos que utiliza valores de la lista desplegable.
    filtro = (df['start station name'] == seleccion) 
    # Filtramos el DataFrame.
    df = df.loc[filtro]
    # Agrupamos los datos por estación de llegada para poder contrarlos.
    df=df.groupby('end station name').count().reset_index()
    # Ordenamos los datos de mayor a menor.
    df.sort_values(by=['bikeid'], inplace=True, ascending=False)
    
    # Definimos y retornamos la figura.
    fig = go.Figure(go.Bar(
                            x=df['bikeid'],
                            y=df["end station name"],
                            text=df['bikeid'],
                            textposition='outside',                                # Posición de la etiqueta de datos. En este caso a la derecha.
                            texttemplate='%{text}',                                # Formato de texto.
                            marker_color='rgb(26, 118, 255)',                      # Color de las barras.
                            orientation='h'                                        # Orientación de las barras. En este caso horizontal.
                          )
                   )
    
                    
    fig.update_layout(
                        barmode='stack',                                           # Tipo de barras. En este caso apiliadas.
                        yaxis_title="Estación de llegada",
                        xaxis_title="Número de viajes",
                        title= 'Rutas populares saliendo de '+ str(seleccion),
                        xaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje x.
                        yaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje y.
                        plot_bgcolor='white',                                      # Modificamos el color del fondo del gráfico.
                        showlegend=False,                                          # Ocultamos la leyenda.
                        yaxis_range=[9.5, -.5]                                     # Mostramos los 10 primeros datos.
                     )
    
    return fig

Declaramos el tablero de control.

In [7]:
tablero_control = pn.Row(pn.Column(titulo, seleccion), funcion_interactiva)

Finalmente, mostramos el tablero de control.

In [8]:
tablero_control.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='Estación de salida', options=['Underhill Ave & Lincoln ...], value='Underhill Ave &...)
    [1] ParamFunction(function)

Si quisieramos ver el tablero de control en otra ventana del navegador, podríamos ejecutar alguno de los siguientes comandos:
```python
tablero_control.show()
```

o

```python
pn.serve(tablero_control)
```

Esto solo funciona si tenemos acceso a los puertos de servidor del computador que está ejecutando la aplicación (`panel`). Puedes intentarlo descargando este notebook y ejecutándolo en tu computador personal.

## Referencias

Panel (2021). A high-level app and dashboarding solution for Python. Recuperado el 11 de febrero de 2021 de: https://panel.holoviz.org/

Citi Bike (2021). System Data. Recuperado el 11 de febrero de 2021 de: https://www.citibikenyc.com/system-data

## Créditos

**Autores:** Juan David Reyes Jaimes, Alejandro Mantilla Redondo,Diego Alejandro Cely Gómez

**Fecha última actualización:** 27/07/2022